In [41]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import math
import warnings
warnings.filterwarnings('ignore')

filename1 = 'blood_train.csv'
filename2 = 'blood_test.csv'
data1 = np.loadtxt(filename1, delimiter=',')
data2 = np.loadtxt(filename2, delimiter=',')
x_train = data1[:,0:-1]
y_train = data1[:,-1]
x_test = data2[:,0:-1]
y_test = data2[:,-1]

#Implement sklearn 
sklearn_imp = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=10)
sklearn_imp.fit(x_train,y_train)

y_train_pre = sklearn_imp.predict(x_train)
sk_trerror = sum(y_train_pre!= y_train) / len(y_train)
print('Training error by using sklearn adaboost: ', sk_trerror)

y_test_pre = sklearn_imp.predict(x_test)
sk_tserror = sum(y_test_pre!= y_test) / len(y_test)
print('Testing error by using sklearn adaboost: ', sk_tserror)

#Self adaboost algorithm
def sample(x, y, D, n):
    X = []
    Y = []
    for i in range(n):
        num = int(np.ceil(D[i]*n)) 
        for j in range(num):
            X.append(x[i,:])
            Y.append(y[i])
    return np.matrix(X)[0:n,:], np.array(Y)[0:n]

def classify(data):
    n=len(data)
    label=[]
    for i in range (n):
        if(data[i]>0.0):
            label.append(1.0)
        else:
            label.append(0.0)
    return label

def prob_distribution(et, at, y, htx, D):
    zt = 2.0* math.sqrt(et*(1-et))#normalization constant
    size_d = len(D)
    new_d = np.zeros(size_d)
    for i in range (size_d):
        new_d[i] = ((D[i]/zt) * math.exp((-at)* y[i] * htx[i])) / np.sum(new_d)
    return new_d

class adaboost():
    def __init__(self, rounds=20, weak_clf = []):
        self.rounds=rounds
        self.H = []
        self.at = []
    
    def fit(self, x, y):
        row, col = np.shape(x)
        D = (np.ones((row))) / row
        np.shape(D)
        weak_clf = DecisionTreeClassifier(max_depth=2)
        y = np.transpose(y)

        for t in range (self.rounds): 
            xs, ys = sample(x_train, y_train, D, row)
            weak_clf.fit(xs, ys)
            htx = weak_clf.predict(xs)
            error_h = np.sum(np.abs(np.subtract(y_train,htx))) / len(y_train)
            size_d = len(D)
            mistake = np.abs(np.subtract(y, htx))
            error = 0.0
        
            for i in range (size_d):
                total_error = error + (D[i]*mistake[i])
                alpha_t = 0.5 * math.log10((1-total_error)/ total_error)
                self.at.append(alpha_t)
                new_prob_distribution = prob_distribution(total_error, alpha_t, y, htx, D)
                self.H.append(weak_clf)
        
    def predict(self,x):
        row, col = np.shape(x)
        y_pred = np.zeros(row)
        for clf, a in zip(self.H,self.at):
            y = clf.predict(x)
            y_pred = y_pred + (a * y)
        y_pred = classify(y_pred)
        return y_pred
    
adaboost = adaboost(rounds=20)
adaboost.fit(x_train, y_train)
y_train_pred = adb.predict(x_train)    
y_test_pred = adb.predict(x_test)
training_error = sum(y_train_pred != y_train) / len(y_train)
testing_error = sum(y_test_pred != y_test) / len(y_test)

print('Training error by using self adaboost: ', training_error)
print('Testing error by using self adaboost: ', testing_error)    


Training error by using sklearn adaboost:  0.17725752508361203
Testing error by using sklearn adaboost:  0.29333333333333333
Training error by using self adaboost:  0.22742474916387959
Testing error by using self adaboost:  0.32
